In [1]:
import pandas as pd
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [2]:
# df = pd.read_csv('eda.csv', low_memory= False)
df_1 = pd.read_csv('eda_part_1.csv', low_memory= False)
df_2 = pd.read_csv('eda_part_2.csv', low_memory= False)
df_3 = pd.read_csv('eda_part_3.csv', low_memory= False)
df_4 = pd.read_csv('eda_part_4.csv', low_memory= False)
df = pd.concat([df_1, df_2, df_3, df_4], ignore_index= True)
df.head()

,Unnamed: 0,provincia_iso,sexo,grupo_edad,fecha,num_casos,num_hosp,num_uci,num_def,country_region_code,...,iso_3166_2_code,place_id,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,provincia_iso_g
0,0,A,H,0-9,2020-01-01,0,0,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,A,H,10-19,2020-01-01,0,0,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,A,H,20-29,2020-01-01,0,0,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,A,H,30-39,2020-01-01,0,0,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,A,H,40-49,2020-01-01,0,0,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df = df.drop('Unnamed: 0', axis=1)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1228800 entries, 0 to 1228799
Data columns (total 22 columns):
 #   Column                                              Non-Null Count    Dtype  
---  ------                                              --------------    -----  
 0   provincia_iso                                       1228800 non-null  object 
 1   sexo                                                1228800 non-null  object 
 2   grupo_edad                                          1228800 non-null  object 
 3   fecha                                               1228800 non-null  object 
 4   num_casos                                           1228800 non-null  int64  
 5   num_hosp                                            1228800 non-null  int64  
 6   num_uci                                             1228800 non-null  int64  
 7   num_def                                             1228800 non-null  int64  
 8   country_region_code                                 

In [5]:
primer_dato = df['sub_region_2'].first_valid_index()
primer_dato

70200

In [6]:
df = df.iloc[primer_dato:]

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1158600 entries, 70200 to 1228799
Data columns (total 22 columns):
 #   Column                                              Non-Null Count    Dtype  
---  ------                                              --------------    -----  
 0   provincia_iso                                       1158600 non-null  object 
 1   sexo                                                1158600 non-null  object 
 2   grupo_edad                                          1158600 non-null  object 
 3   fecha                                               1158600 non-null  object 
 4   num_casos                                           1158600 non-null  int64  
 5   num_hosp                                            1158600 non-null  int64  
 6   num_uci                                             1158600 non-null  int64  
 7   num_def                                             1158600 non-null  int64  
 8   country_region_code                             

In [8]:
filas_nulas = df[df['sub_region_2'].isnull()]
a = list(filas_nulas['provincia_iso'].unique())
b = list(df['provincia_iso_g'].unique())

In [9]:
def find_non_null_indices(dataframe, col1, col2):
    """
    Encuentra índices de filas donde el valor en col1 es nulo y el valor en col2 no es nulo.

    :param dataframe: DataFrame en el que buscar.
    :param col1: Nombre de la primera columna para comprobar si es nulo.
    :param col2: Nombre de la segunda columna para comprobar si no es nulo.
    :return: Lista de índices de filas que cumplen la condición.
    """
    indices = []
    for index, row in dataframe.iterrows():
        if pd.isna(row[col1]) and not pd.isna(row[col2]):
            indices.append(index)
    return indices

In [10]:
find_non_null_indices(df, 'provincia_iso', 'provincia_iso_g')       # Con esto con cluimos que todas la filas con nulos en provincias_iso son también nulas en provincias_iso_g, pero y al revés?

[]

In [11]:
no_coinc = find_non_null_indices(df, 'provincia_iso_g', 'provincia_iso')        # Vemos que en este sentido sí hay alguno que no cuadran

In [12]:
len(no_coinc)

390

In [13]:
sum(df['provincia_iso'].notna()) - sum(df['provincia_iso_g'].notna())           # Vemos que el desfase entre estas dos columnas se corresponde con este hecho. Vamos a intentar averiguar a qué se debe. 

390

In [14]:
def unique_values_in_rows(df, indices, columns = None):
    """
    Muestra los valores únicos en las filas especificadas para las columnas dadas.

    :param df: DataFrame de Pandas.
    :param indices: Lista de índices de las filas.
    :param columns: Lista de columnas para mostrar valores únicos.
    :return: Un diccionario con los valores únicos para cada columna.
    """
    if columns is None:
        columns = df.columns
    unique_values = {}
    for column in columns:
        # Filtrar por los índices dados y luego extraer valores únicos para la columna
        unique_values[column] = df.loc[indices, column].unique()
    return unique_values

In [15]:
pp.pprint(unique_values_in_rows(df, no_coinc))          # Como vemos, para Ceuta y Melilla, hay 7 días en los que no hay datos, comprobado que efectivamente es así en el df original de google, y que no ha sido un fallo del merge tomamos una decisión. 
                                                        # lo suyo es que rellenásemos los valores, pero para hacerlo de la manera más óptima tendríamos que entrenar un modelo. Así que lo vamos a hacer es simplemente prescindir de ceuta y melilla para este EDA.
                                                        # Ya que además en el resto de días también hay muchos huecos.

{   'country_region': array([nan], dtype=object),
    'country_region_code': array([nan], dtype=object),
    'date': array([nan], dtype=object),
    'fecha': array(['2020-08-22', '2020-08-23', '2020-08-29', '2020-08-30',
       '2020-09-02', '2020-09-05', '2020-09-06'], dtype=object),
    'grocery_and_pharmacy_percent_change_from_baseline': array([nan]),
    'grupo_edad': array(['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69',
       '70-79', '80+', 'NC'], dtype=object),
    'iso_3166_2_code': array([nan], dtype=object),
    'num_casos': array([0, 1, 2, 4, 7, 3], dtype=int64),
    'num_def': array([0], dtype=int64),
    'num_hosp': array([0, 1, 2], dtype=int64),
    'num_uci': array([0], dtype=int64),
    'parks_percent_change_from_baseline': array([nan]),
    'place_id': array([nan], dtype=object),
    'provincia_iso': array(['CE', 'ML'], dtype=object),
    'provincia_iso_g': array([nan], dtype=object),
    'residential_percent_change_from_baseline': array([nan]),
    'ret

In [16]:
df = df[(df['provincia_iso'] != 'CE') & (df['provincia_iso'] != 'ML')]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1114020 entries, 70200 to 1228799
Data columns (total 22 columns):
 #   Column                                              Non-Null Count    Dtype  
---  ------                                              --------------    -----  
 0   provincia_iso                                       1114020 non-null  object 
 1   sexo                                                1114020 non-null  object 
 2   grupo_edad                                          1114020 non-null  object 
 3   fecha                                               1114020 non-null  object 
 4   num_casos                                           1114020 non-null  int64  
 5   num_hosp                                            1114020 non-null  int64  
 6   num_uci                                             1114020 non-null  int64  
 7   num_def                                             1114020 non-null  int64  
 8   country_region_code                                 1

In [17]:
pp.pprint(unique_values_in_rows(df,df[df['provincia_iso'].isna()].index.tolist()))

{   'country_region': array([], dtype=object),
    'country_region_code': array([], dtype=object),
    'date': array([], dtype=object),
    'fecha': array([], dtype=object),
    'grocery_and_pharmacy_percent_change_from_baseline': array([], dtype=float64),
    'grupo_edad': array([], dtype=object),
    'iso_3166_2_code': array([], dtype=object),
    'num_casos': array([], dtype=int64),
    'num_def': array([], dtype=int64),
    'num_hosp': array([], dtype=int64),
    'num_uci': array([], dtype=int64),
    'parks_percent_change_from_baseline': array([], dtype=float64),
    'place_id': array([], dtype=object),
    'provincia_iso': array([], dtype=object),
    'provincia_iso_g': array([], dtype=object),
    'residential_percent_change_from_baseline': array([], dtype=float64),
    'retail_and_recreation_percent_change_from_baseline': array([], dtype=float64),
    'sexo': array([], dtype=object),
    'sub_region_1': array([], dtype=object),
    'sub_region_2': array([], dtype=object),
    '

In [18]:
len(df[df['provincia_iso'].isna()].index.tolist())

0

In [19]:
pp.pprint(unique_values_in_rows(df,df[df['transit_stations_percent_change_from_baseline'].isna()].index.tolist()))

{   'country_region': array(['Spain'], dtype=object),
    'country_region_code': array(['ES'], dtype=object),
    'date': array(['2020-02-22', '2020-03-28', '2020-03-29', '2020-04-04',
       '2020-04-05', '2020-04-09', '2020-04-10', '2020-04-11',
       '2020-04-19', '2020-08-17', '2020-08-18', '2020-08-19',
       '2020-08-20', '2020-08-21', '2020-08-22', '2020-08-23',
       '2020-08-24', '2020-08-25', '2020-08-26', '2020-08-27',
       '2020-08-28', '2020-08-29', '2020-08-30', '2020-08-31',
       '2020-09-01', '2020-09-02', '2020-09-03', '2020-09-04',
       '2020-09-05', '2020-09-06', '2020-09-07', '2020-09-08',
       '2020-09-09', '2020-09-10'], dtype=object),
    'fecha': array(['2020-02-22', '2020-03-28', '2020-03-29', '2020-04-04',
       '2020-04-05', '2020-04-09', '2020-04-10', '2020-04-11',
       '2020-04-19', '2020-08-17', '2020-08-18', '2020-08-19',
       '2020-08-20', '2020-08-21', '2020-08-22', '2020-08-23',
       '2020-08-24', '2020-08-25', '2020-08-26', '2020-08-

In [20]:
def fillna_valor_mas_cercano(df, fillcol, fecha_col, provincia_col):
    '''
    Rellena los valores nulos de una columna en función del valor más cercano en días
    para una provincia dada.
    
    :param df: DataFrame
    :param fillcol: Nombre de la columna a rellenar.
    :param fecha_col: Nombre de la columna de fecha.
    :param provincia_col: Nombre de la columna de provincia.
    '''
    # Asegurarse de que la columna de fecha sea de tipo datetime
    df[fecha_col] = pd.to_datetime(df[fecha_col])

    # Obtener las provincias únicas
    # Obtener los índices de las filas donde 'fillcol' es nulo
    indices_nulos = df[df[fillcol].isna()].index.tolist()

    # Usar la función unique_values_in_rows para obtener los valores únicos de 'provincia_col'
    valores_unicos = unique_values_in_rows(df, indices_nulos, [provincia_col])

    # Extraer las provincias únicas desde el diccionario
    provincias = valores_unicos[provincia_col]


    for provincia in provincias:
        # Filtrar el DataFrame por provincia
        df_prov = df[df[provincia_col] == provincia].sort_values(fecha_col)

        for index, row in df_prov.iterrows():
            if pd.isna(row[fillcol]):
                fecha = row[fecha_col]
                dias = 1
                valor_encontrado = False

                while not valor_encontrado:
                    # Buscar en días anteriores y posteriores
                    dia_anterior = fecha - pd.Timedelta(days=dias)
                    dia_posterior = fecha + pd.Timedelta(days=dias)

                    # Valores en días adyacentes
                    valor_anterior = df_prov[df_prov[fecha_col] == dia_anterior][fillcol].dropna()
                    valor_posterior = df_prov[df_prov[fecha_col] == dia_posterior][fillcol].dropna()

                    if not valor_anterior.empty:
                        df.at[index, fillcol] = valor_anterior.iloc[0]
                        valor_encontrado = True
                    elif not valor_posterior.empty:
                        df.at[index, fillcol] = valor_posterior.iloc[0]
                        valor_encontrado = True

                    dias += 1  # Incrementar el rango de búsqueda

In [21]:
add_data_col = ['retail_and_recreation_percent_change_from_baseline', 'grocery_and_pharmacy_percent_change_from_baseline', 'parks_percent_change_from_baseline', 'transit_stations_percent_change_from_baseline', 'workplaces_percent_change_from_baseline', 'residential_percent_change_from_baseline']
for col in add_data_col:
    fillna_valor_mas_cercano(df, col, 'fecha', 'provincia_iso')

In [22]:
df.loc[df['sub_region_2'] == 'Huelva', 'transit_stations_percent_change_from_baseline'].unique()

array([ 45.,  18.,  16.,   8.,  11.,  34.,  49.,  29.,  28.,  17.,  15.,
        22.,  31.,  14.,  13.,  10.,  32.,  44.,  25.,  19.,  12.,  -2.,
       -32., -66., -58., -65., -64., -68., -77., -82., -69., -73., -72.,
       -70., -76., -85., -71., -78., -81., -74., -83., -67., -75., -62.,
       -63., -54., -43., -45., -47., -59., -31., -40., -49., -42., -46.,
       -28., -35., -34., -36., -21., -24., -25., -23., -18., -17., -26.,
       -27., -19., -12., -16., -13.,  -5., -15., -20., -10.,   6.,  -8.,
        -3.,  -7.,  -1.,   3.,   5.,   7.,  20.,  35.,  36.,   9.,  24.,
        23.,  47.,  38.,  40.,  37.,  43.,  48.,   2.,  30.,   0.,  26.,
        21.,  -4., -14.,  -9.,   1.,  -6.,   4., -37., -11., -33., -38.,
       -30., -39.,  27.,  33.,  46.,  53.,  52.,  39.,  54.,  42.,  57.,
        56.,  41.,  61.,  51.,  50.,  74.,  68.,  62.,  63.,  60.,  67.,
        58.,  73.,  84.,  64.,  93.,  99.,  96., 105.,  87., 106., 100.,
        95., 104., 107.,  85., 103.,  97.,  91.,  7

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1114020 entries, 70200 to 1228799
Data columns (total 22 columns):
 #   Column                                              Non-Null Count    Dtype         
---  ------                                              --------------    -----         
 0   provincia_iso                                       1114020 non-null  object        
 1   sexo                                                1114020 non-null  object        
 2   grupo_edad                                          1114020 non-null  object        
 3   fecha                                               1114020 non-null  datetime64[ns]
 4   num_casos                                           1114020 non-null  int64         
 5   num_hosp                                            1114020 non-null  int64         
 6   num_uci                                             1114020 non-null  int64         
 7   num_def                                             1114020 non-null  int

In [24]:
df['place_id'].unique()

array(['ChIJfdrHtbc5Yg0RYMkh126vAgM', 'ChIJ0XrEWB18Zg0RUMlLvvNAAwM',
       'ChIJZQoW955UZQ0RcMl1pXjSAwM', 'ChIJj6yPsjuEQA0RgMnzWgeHBQM',
       'ChIJZb1_yQvmpBIRsMmjIeD6AAM', 'ChIJ28y339ptFA0RkMmfjP1jBAM',
       'ChIJzxBUoQZOTg0RMMzJc4L1BAM', 'ChIJ9ZsJuUntRQ0RwMnzWgeHBQM',
       'ChIJGQVeRvRfLg0RIMrJc4L1BAM', 'ChIJaxjPWk6qDQ0R4MmfjP1jBAM',
       'ChIJEz0HKNv5FQ0R0MmfjP1jBAM', 'ChIJm30-GKPebA0REMp1pXjSAwM',
       'ChIJ6_wEDsndaw0RAMpLvvNAAwM', 'ChIJd-4zz4vzXw0R8Mkh126vAgM',
       'ChIJPaBPpUf1Zw0RMMpLvvNAAwM', 'ChIJvy1D_6YfRwwRYMtLvvNAAwM',
       'ChIJRRrTHsPNuhIRQMqjIeD6AAM', 'ChIJuexUhnolbg0RUMp1pXjSAwM',
       'ChIJl9C4p3cuQw0RYMpLvvNAAwM', 'ChIJiYiy9MW-EQ0RgMqfjP1jBAM',
       'ChIJtwOaTjZxWA0RkMrNCGWMAQM', 'ChIJxaS9-r71bg0RoMp1pXjSAwM',
       'ChIJJQQpq4R6phIRwMqjIeD6AAM', 'ChIJU7YQYQduNw0RsMrzWgeHBQM',
       'ChIJu9z3TpJLRQ0Rjimbpl76f3g', 'ChIJU82ZLkW2MQ0R4MrJc4L1BAM',
       'ChIJuTPgQHqBQQ0RgMhLvvNAAwE', 'ChIJUyii_DXqcg0RAMt1pXjSAwM',
       'ChIJza6U13waYw0RIthUR_Uzzh

In [25]:
df.drop(['date', 'provincia_iso_g', 'country_region_code', 'iso_3166_2_code'], axis=1, inplace= True)

In [26]:
df.rename(columns = {'country_region': 'país', 'sub_region_1': 'comunidad', 'sub_region_2': 'provincia', 'retail_and_recreation_percent_change_from_baseline': 'comercio_ocio', 'grocery_and_pharmacy_percent_change_from_baseline': 'supermercados_farmacias',
                     'parks_percent_change_from_baseline': 'parques', 'transit_stations_percent_change_from_baseline': 'estaciones', 'workplaces_percent_change_from_baseline': 'trabajo', 'residential_percent_change_from_baseline': 'residencia'}, inplace= True)

In [27]:
nuevo_orden = ['país', 'comunidad', 'provincia', 'provincia_iso', 'fecha', 'sexo', 'grupo_edad', 'num_casos', 'num_hosp', 'num_uci', 'num_def', 'place_id',
       'comercio_ocio', 'supermercados_farmacias', 'parques', 'estaciones', 'trabajo', 'residencia']
df = df[nuevo_orden]

In [28]:
df.reset_index(drop = True, inplace= True)
df.head()

,país,comunidad,provincia,provincia_iso,fecha,sexo,grupo_edad,num_casos,num_hosp,num_uci,num_def,place_id,comercio_ocio,supermercados_farmacias,parques,estaciones,trabajo,residencia
0,Spain,Valencian Community,Alicante,A,2020-02-15,H,0-9,0,0,0,0,ChIJfdrHtbc5Yg0RYMkh126vAgM,3.0,-1.0,34.0,7.0,0.0,-1.0
1,Spain,Valencian Community,Alicante,A,2020-02-15,H,10-19,0,0,0,0,ChIJfdrHtbc5Yg0RYMkh126vAgM,3.0,-1.0,34.0,7.0,0.0,-1.0
2,Spain,Valencian Community,Alicante,A,2020-02-15,H,20-29,0,0,0,0,ChIJfdrHtbc5Yg0RYMkh126vAgM,3.0,-1.0,34.0,7.0,0.0,-1.0
3,Spain,Valencian Community,Alicante,A,2020-02-15,H,30-39,0,0,0,0,ChIJfdrHtbc5Yg0RYMkh126vAgM,3.0,-1.0,34.0,7.0,0.0,-1.0
4,Spain,Valencian Community,Alicante,A,2020-02-15,H,40-49,0,0,0,0,ChIJfdrHtbc5Yg0RYMkh126vAgM,3.0,-1.0,34.0,7.0,0.0,-1.0


In [29]:
df.to_csv('eda_estudio.csv')

In [30]:
chunksize = 300 * 1024  # Tamaño máximo por fragmento (25MB)
i = 1

for chunk in pd.read_csv('eda_estudio.csv', chunksize=chunksize, low_memory= False):
    chunk.to_csv(f'eda_estudio_part_{i}.csv', index=False)
    i += 1